<a href="https://colab.research.google.com/github/CampbellAgreev/Analysis-of-housing-information/blob/master/Faster_R_CNN_and_YOLOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
project_dir = "/content/drive/MyDrive/Project Folder"
os.chdir(project_dir)
!git clone https://github.com/duckrabbits/ObjectDetection.git
%cd ObjectDetection
!mv ../simpsons_dataset ./simpsons_dataset
!mv ../annotation.txt ./annotation.txt


Mounted at /content/drive
fatal: destination path 'ObjectDetection' already exists and is not an empty directory.
/content/drive/MyDrive/Project Folder/ObjectDetection
mv: cannot stat '../simpsons_dataset': No such file or directory
mv: cannot stat '../annotation.txt': No such file or directory


In [2]:
!pip install tensorflow keras opencv-python-headless scikit-image h5py matplotlib


In [3]:

!python train.py -p annotation.txt -E 1


2025-05-25 23:26:10.078371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748215570.145446    1237 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748215570.165644    1237 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-25 23:26:10.254331: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Using dataset from: /content/drive/MyDrive/Project Folder/ObjectDetection/simpsons_dataset
Training set: (14317, 64, 

1.Backbone (ResNet)
File: model_frcnn.py

Function: nn_base(input_tensor=None, trainable=False)

Purpose: Builds the convolutional layers (e.g., VGG16 without FC layers).


from keras.applications.vgg16 import VGG16

def nn_base(input_tensor=None, trainable=False):
    # Load pretrained VGG16 without top
    base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
    # Set layer trainability
    for layer in base_model.layers:
        layer.trainable = trainable
    return base_model.output

 2.Region Proposal Network (RPN)
File: model_frcnn.py

Function: rpn(base_layers, num_anchors)

Purpose: Adds RPN layers (classification + regression for anchors).


##def rpn(base_layers, num_anchors):
    x = Conv2D(512, (3,3), padding='same', activation='relu')(base_layers)
    x_class = Conv2D(num_anchors, (1,1), activation='sigmoid')(x)
    x_regr = Conv2D(num_anchors * 4, (1,1), activation='linear')(x)
    return [x_class, x_regr, base_layers]

3.RoI Pooling Layer
File: roi_helpers.py or layers.py

Function: RoiPoolingConv class (custom layer)

Purpose: Converts each RoI to a fixed size feature map.

##class RoiPoolingConv(Layer):
    def __init__(self, pool_size, num_rois, **kwargs):
        Custom RoI pooling layer

4.Classifier & Regressor (Head)
File: model_frcnn.py

Function: classifier_layers(x, input_rois, num_rois, nb_classes)

Purpose: Performs classification and bounding box regression on pooled RoIs.

##def classifier_layers(base_layers, input_rois, num_rois, nb_classes=21):
    Apply dense layers, softmax for class, linear for bbox regression

5.Anchor Target Assignment, Loss Functions, Training Loop
Files:

losses.py: Contains rpn_loss_cls, rpn_loss_regr, class_loss_cls, class_loss_regr

train_frcnn.py: Main training loop

data_generators.py: Anchor labeling, IoU computation


In [4]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
project_dir = "/content/drive/MyDrive/Project Folder"
os.chdir(project_dir)

if not os.path.exists("keras-yolo3"):
    !git clone https://github.com/qqwweee/keras-yolo3.git

os.chdir("keras-yolo3")

cfg_path = "/content/drive/MyDrive/Project Folder/yolov3.cfg"
if os.path.exists(cfg_path):
    !cp "{cfg_path}" yolov3.cfg
else:
    raise FileNotFoundError(f"Specified YOLOv3 config file not found: {cfg_path}")
weights_path = "/content/drive/MyDrive/Project Folder/yolov3 (1).weights"
if os.path.exists(weights_path):
    !cp "{weights_path}" yolov3.weights
else:
    raise FileNotFoundError(f"Specified weights file not found: {weights_path}")

!find . -type f -name "*.py" -exec sed -i 's/from keras/from tensorflow.keras/g' {} +
!find . -type f -name "*.py" -exec sed -i 's/import keras/import tensorflow.keras/g' {} +
!find . -type f -name "*.py" -exec sed -i '/multi_gpu_model/d' {} +

!python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5

if not os.path.exists("model_data/yolo.h5"):
    raise FileNotFoundError("Model conversion failed! 'model_data/yolo.h5' was not created.")

!python yolo_video.py --image --model model_data/yolo.h5


Mounted at /content/drive
2025-05-26 00:06:43.707768: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748218003.729998   34975 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748218003.736752   34975 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-26 00:06:43.769505: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Loading weights.
Weights Header: 0 2 0 [32013312]
Parsing Darknet config.
Creating Keras mo

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os


os.chdir("/content/drive/MyDrive/Project Folder/SimpsonRecognition-master")


input_file = "annotation.txt"
output_file = "train_annotations.txt"
classes_file = "simpsons_classes.txt"


base_image_dir = "/content/drive/MyDrive/Project Folder/ObjectDetection/simpsons_dataset"


class_names = set()
with open(input_file, "r") as f:
    for line in f:
        parts = line.strip().split(',')
        if len(parts) == 6:
            class_names.add(parts[5])

class_list = sorted(class_names)
class_to_id = {name: idx for idx, name in enumerate(class_list)}


with open(classes_file, "w") as f:
    for cls in class_list:
        f.write(f"{cls}\n")


image_to_boxes = {}
with open(input_file, "r") as f:
    for line in f:
        parts = line.strip().split(',')
        if len(parts) != 6:
            continue
        image_path, x1, y1, x2, y2, cls_name = parts


        if image_path.startswith('./characters/'):
            image_path = image_path[len('./characters/'):]


        abs_path = os.path.abspath(os.path.join(base_image_dir, image_path))


        if not os.path.exists(abs_path):
            print(f"Warning: Missing image file {abs_path}")
            continue

        class_id = class_to_id[cls_name]
        box = f"{x1},{y1},{x2},{y2},{class_id}"
        if abs_path not in image_to_boxes:
            image_to_boxes[abs_path] = []
        image_to_boxes[abs_path].append(box)


with open(output_file, "w") as f:
    for img_path, boxes in image_to_boxes.items():
        f.write(f"{img_path} {' '.join(boxes)}\n")

print(f"✔ Saved {len(class_list)} classes to: {os.path.abspath(classes_file)}")
print(f"✔ Annotations saved to: {os.path.abspath(output_file)}")
print(f"✔ Total annotated images: {len(image_to_boxes)}")

%cd "/content/drive/MyDrive/Project Folder/keras-yolo3"
!python train.py


Mounted at /content/drive
✔ Saved 18 classes to: /content/drive/MyDrive/Project Folder/SimpsonRecognition-master/simpsons_classes.txt
✔ Annotations saved to: /content/drive/MyDrive/Project Folder/SimpsonRecognition-master/train_annotations.txt
✔ Total annotated images: 7889
/content/drive/MyDrive/Project Folder/keras-yolo3
2025-05-30 00:04:36.361697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748563476.381751    1770 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748563476.387613    1770 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-30 00:04:36.406591: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optim

1.Backbone Network: Darknet-53

YOLOv3 utilizes Darknet-53, a 53-layer convolutional network with residual connections, for feature extraction.

Code Reference:
The backbone is implemented in the yolo_body function within the yolo3.model module, which is invoked here:

##model_body = yolo_body(image_input, num_anchors//3, num_classes)

This constructs the core YOLOv3 model architecture.

2.Multi-Scale Predictions

YOLOv3 predicts bounding boxes at three different scales to detect objects of varying sizes.

Code Reference:
The model outputs predictions at three scales, corresponding to different feature map sizes. The loss function combines these outputs:

#model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
                     arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})([*model_body.output, *y_true])

Here, y_true contains ground truth tensors for each scale, and model_body.output provides the corresponding predictions.

3.Anchor Boxes

YOLOv3 uses predefined anchor boxes to predict bounding boxes
Code Reference:
Anchor boxes are loaded and reshaped as follows:

#def get_anchors(anchors_path):
    with open(anchors_path) as f:
        anchors = f.readline()
    return np.array([float(x) for x in anchors.split(',')]).reshape(-1, 2)

These anchors are then utilized in the loss function to guide bounding box predictions.

4.Loss Function: YOLO Loss

The loss function in YOLOv3 comprises multiple components, including localization, confidence, and classification losses.

Code Reference:
The custom loss function is applied using a Lambda layer:

#model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
                     arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})([*model_body.output, *y_true])
This setup allows the model to compute the combined loss during training.

5.Training Strategy: Layer Freezing and Fine-Tuning

To stabilize training, initial layers are frozen, and later all layers are unfrozen for fine-tuning.

Code Reference:
Layers are frozen based on the freeze_body parameter:

#if freeze_body in [1, 2]:
    num = (185, len(model_body.layers)-3)[freeze_body-1]
    for i in range(num):
        model_body.layers[i].trainable = False
    print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))
Later, all layers are unfrozen for further training:

#for layer in model.layers:
    layer.trainable = True

6.Data Augmentation

Data augmentation techniques are applied to enhance model generalization.

Code Reference:
The get_random_data function applies random transformations to the input images:

#image, box = get_random_data(annotation_lines[i], input_shape, random=True)

7.raining Pipeline: Data Generators and Model Fitting

The training process utilizes data generators and the Keras fit method.

Code Reference:
Data generators are defined as:

train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
    output_signature=get_output_signature(input_shape, anchors, num_classes, batch_size)
)

The model is trained using:

model.fit(
    train_dataset,
    steps_per_epoch=max(1, num_train // batch_size),
    validation_data=val_dataset,
    validation_steps=max(1, num_val // batch_size),
    epochs=50,
    callbacks=[logging, checkpoint]
)

8.Optimization: Learning Rate Scheduling and Early Stopping

To improve training efficiency, learning rate reduction and early stopping are implemented.

Code Reference:
Callbacks are set up as follows:

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

These callbacks adjust the learning rate when the validation loss plateaus and stop training when no improvement is observed.
